# 第7章 函数修饰器和闭包

## 实例：修饰器

In [2]:
import time

In [14]:
def time_consume(fun):
    """定义一个装饰器"""
    def wrapper():
        t1 = time.time()
        fun()
        print(time.time() - t1)
    return wrapper

In [4]:
def fun1():
    t1 = time.time()
    print("您好")
    print(time.time() - t1)

In [5]:
def fun2():
    print("北京")

In [6]:
@time_consume
def fun3():
    count = 0
    for i in range(10000):
        count += 1
    print(count)

In [8]:
# 返回值不在是fun3函数
fun3

<function __main__.time_consume.<locals>.wrapper()>

10000
0.0019364356994628906


In [10]:
fun2()

北京


In [11]:
fun2

<function __main__.fun2()>

In [12]:
fun1()

您好
8.511543273925781e-05


In [13]:
fun1

<function __main__.fun1()>

## 装饰器的执行顺序
装饰器的一个关键特性是，它们在被装饰的函数定义之后立即运行。这通常是在导入时（即Python加载模块时）， 而为修饰的函数需要在触发后在执行。

In [15]:
# BEGIN REGISTRATION

registry = []  # <1>

def register(func):  # <2>
    print('running register(%s)' % func)  # <3>
    registry.append(func)  # <4>
    return func  # <5>

@register  # <6>
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():  # <7>
    print('running f3()')

def main():  # <8>
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__=='__main__':
    main()  # <9>

# END REGISTRATION

running register(<function f1 at 0x7f1222f829d0>)
running register(<function f2 at 0x7f123947cd30>)
running main()
registry -> [<function f1 at 0x7f1222f829d0>, <function f2 at 0x7f123947cd30>]
running f1()
running f2()
running f3()


In [16]:
import registration

running register(<function f1 at 0x7f1222f82af0>)
running register(<function f2 at 0x7f1222f82f70>)


In [18]:
registration.registry

[<function registration.f1()>, <function registration.f2()>]

## 变量作用域规则
[Python 变量作用域](https://blog.csdn.net/cc7756789w/article/details/46635383)

## 作用域

In [42]:
def f1(a):
    print(a)
    print(b)

In [43]:
f1(a = 1)

1


NameError: name 'b' is not defined

In [44]:
b = 1

In [45]:
f1(a=1)

1
1


In [46]:
# 局部变量未定义
def f2(a):
    print(a)
    print(b)
    b = 5

In [47]:
b = 4
f2(5)

5


UnboundLocalError: local variable 'b' referenced before assignment

In [48]:
# 局部变量未定义
def f3(a):
    global b
    print(a)
    print(b)
    b = 5

In [49]:
f3(4)

4
4


In [50]:
b = 100

In [51]:
f3(4)

4
100


In [52]:
# 局部作用域和全局作用域的区别
# 在 def/class/lambda内进行赋值，就变成了其局部的作用域，局部作用域会覆盖全局作用域，但不会影响全局作用域。
g = 1000

def fun1(g):
    
    g = 100
    
    return g

print(fun1(10))
print(g)

100
1000


## 闭包
概念：其实，闭包指延伸了作用域的函数，其中包含函数定义体中引用、但是不在定义体中定义的非全局变量。函数是不是匿名的没有关系，==关键是它能访问定义体之外定义的非全局变量==。  
[谈谈自己的理解:python中闭包，闭包的实质](https://www.cnblogs.com/s-1314-521/p/9763376.html)

In [53]:
class Averge:

    def __init__(self):

        self.series = []

    def __call__(self, new_value):

        self.series.append(new_value)
        total_val = sum(self.series)
        return total_val / len(self.series)

In [54]:
# 通过使用avg使得实例对象为可调用的对象
agv = Averge()

In [55]:
agv(4)

4.0

In [56]:
agv(5)

4.5

In [57]:
agv(6)

5.0

In [58]:
# 外函数
def Averge1():
    # 自由变量
    series = []
    # 内函数

    def make_averge(x):

        series.append(x)
        total_val = sum(series)

        return total_val / len(series)
    # 返回内函数的引用
    return make_averge

In [59]:
avg = Averge1()

In [60]:
avg

<function __main__.Averge1.<locals>.make_averge(x)>

In [61]:
avg(10)

10.0

In [62]:
avg(45)

27.5

In [63]:
avg.__closure__

(<cell at 0x7f373a5fcd00: list object at 0x7f373ab8e980>,)

In [64]:
avg.__code__.co_freevars

('series',)

In [65]:
avg.__code__.co_varnames

('x', 'total_val')

In [66]:
# 外函数
def Averge1():
    # 自由变量
    count = 0
    total = 0
    # 内函数

    def make_averge(x):
        nonlocal count, total
        count += 1
        total += x

        return count/total
    # 返回内函数的引用
    return make_averge

In [67]:
A = Averge1()
A(10)

0.1

In [68]:
A(20)

0.06666666666666667

In [69]:
A.__closure__[0].cell_contents

2

# 实现一个简单的装饰器
把被装饰的函数替换成新函数，二者接受相同的参数，而且（通常）返回被装饰的函数本该返回的值，同时还会做些额外操作。

In [110]:
import functools

In [115]:
def clock(func):
    functools.wraps(func)

    def clocked(*args, **kwargs):

        t0 = time.perf_counter()
        result = func(*args, **kwargs)  # 闭包
        t1 = time.perf_counter() - t0
        name = func.__name__
        doc = func.__doc__
        arg_str = ",".join(repr(arg) for arg in args)
        print("[%0.8fs] %s  %s(%s) --> %r" % (t1, doc, name, arg_str, result))

        return result

    return clocked  # 返回内部函数的引用

In [116]:
@clock
def snooze(seconds):
    time.sleep(seconds)


@clock
def fib(x):
    return 1 if x < 2 else x*fib(x - 1)

In [117]:
snooze(.44)
fib(5)  # 每调用一次该函数，都将被修饰一次

[0.44056613s] None  snooze(0.44) --> None
[0.00000105s] None  fib(1) --> 1
[0.00007794s] None  fib(2) --> 2
[0.00013735s] None  fib(3) --> 6
[0.00019228s] None  fib(4) --> 24
[0.00024311s] None  fib(5) --> 120


120

In [118]:
# 等价于
fi = clock(fib(5))
fi

[0.00000105s] None  fib(1) --> 1
[0.00157254s] None  fib(2) --> 2
[0.00161405s] None  fib(3) --> 6
[0.00163869s] None  fib(4) --> 24
[0.00166354s] None  fib(5) --> 120


<function __main__.clock.<locals>.clocked(*args, **kwargs)>

In [119]:
fi.__name__

'clocked'

In [120]:
fib(6)

[0.00000044s] None  fib(1) --> 1
[0.00006110s] None  fib(2) --> 2
[0.00008037s] None  fib(3) --> 6
[0.00009723s] None  fib(4) --> 24
[0.00011283s] None  fib(5) --> 120
[0.00012887s] None  fib(6) --> 720


720

In [121]:
from clockdeco import clock


@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)


if __name__ == '__main__':
    # 正常递归情况的数据运行情况
    # fib(1)最多被调用了8次，增加了时间上的开销
    print(fibonacci(6))

[0.00000048s] fibonacci(0) -> 0
[0.00000095s] fibonacci(1) -> 1
[0.00012970s] fibonacci(2) -> 1
[0.00000048s] fibonacci(1) -> 1
[0.00000095s] fibonacci(0) -> 0
[0.00000048s] fibonacci(1) -> 1
[0.00004673s] fibonacci(2) -> 1
[0.00009584s] fibonacci(3) -> 2
[0.00027132s] fibonacci(4) -> 3
[0.00000048s] fibonacci(1) -> 1
[0.00000048s] fibonacci(0) -> 0
[0.00000048s] fibonacci(1) -> 1
[0.00004268s] fibonacci(2) -> 1
[0.00008655s] fibonacci(3) -> 2
[0.00000048s] fibonacci(0) -> 0
[0.00000048s] fibonacci(1) -> 1
[0.00004292s] fibonacci(2) -> 1
[0.00000024s] fibonacci(1) -> 1
[0.00000072s] fibonacci(0) -> 0
[0.00000048s] fibonacci(1) -> 1
[0.00004387s] fibonacci(2) -> 1
[0.00008774s] fibonacci(3) -> 2
[0.00017190s] fibonacci(4) -> 3
[0.00030231s] fibonacci(5) -> 5
[0.00061679s] fibonacci(6) -> 8
8


In [122]:
@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)


if __name__ == '__main__':
    # 使用lru，节省的时间开销
    print(fibonacci(6))

[0.00000072s] fibonacci(0) -> 0
[0.00000191s] fibonacci(1) -> 1
[0.00132489s] fibonacci(2) -> 1
[0.00000119s] fibonacci(3) -> 2
[0.00168562s] fibonacci(4) -> 3
[0.00000143s] fibonacci(5) -> 5
[0.00196171s] fibonacci(6) -> 8
8


## 参数化修饰器

In [123]:
# BEGIN REGISTRATION_ABRIDGED
registry = []


def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func


@register
def f1():
    print('running f1()')


print('running main()')
print('registry ->', registry)
f1()
# END REGISTRATION_ABRIDGED

running register(<function f1 at 0x7f3739dc5f70>)
running main()
registry -> [<function f1 at 0x7f3739dc5f70>]
running f1()


In [124]:
# BEGIN REGISTRATION_PARAM

registry = set()  # <1>


def register(active=True):  # <2>
    def decorate(func):  # <3>
        print('running register(active=%s)->decorate(%s)'
              % (active, func))
        if active:   # <4>
            registry.add(func)
        else:
            registry.discard(func)  # <5>

        return func  # <6>
    return decorate  # <7>


@register(active=False)  # <8>
def f1():
    print('running f1()')


@register()  # <9>
def f2():
    print('running f2()')


def f3():
    print('running f3()')

# END REGISTRATION_PARAM

running register(active=False)->decorate(<function f1 at 0x7f3739dc5040>)
running register(active=True)->decorate(<function f2 at 0x7f3739dc5f70>)


In [128]:
# 添加修饰器
f2()

running f2()


In [129]:
registry

{<function __main__.f2()>, <function __main__.f3()>}

In [132]:
# 不加修饰器
register()(f3)

running register(active=True)->decorate(<function f3 at 0x7f373a23adc0>)


<function __main__.f3()>

## 通用修饰器

In [134]:
def w_test(func):
    def inner(*args, **kwargs):
        ret = func(*args, **kwargs)
        return ret

    return inner


@w_test
def test():
    print('test called')


@w_test
def test1():
    print('test1 called')
    return 'python'


@w_test
def test2(a):
    print('test2 called and value is %d ' % a)


test()
test1()
test2(9)

# 输出:
# test called
# test1 called
# test2 called and value is 9

test called
test1 called
test2 called and value is 9 


# 参考

[谈谈python修饰器](https://www.jianshu.com/p/ab702e4d4ba7)

